In [1]:
!pip install networkx
!pip install matplotlib

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt
import re
from itertools import combinations

In [ ]:
# Load the card data
with open('data/labeled_cards.json') as file:
    cards = json.load(file)

# Initialize the main graph
G = nx.Graph()

# Add nodes for each card
for card in cards:
    G.add_node(
        card['id'], 
        name=card['name'], 
        labels=card['labels'], 
        traits=card['traits'], 
        effect=card['effect'], 
        cost=card['cost'], 
        color=card['color'], 
        power=card['power'], 
        type=card['type']
    )

In [23]:
def get_searcher_synergy(effect):
    synergy = {}
    synergy['positive'] = re.findall(r'\{([^}]+)\}', effect)
    synergy['negative'] = re.findall(r'\[([^]]+)\]', effect)
    synergy['negative'] = [trait for trait in synergy['negative'] if any(trait in card['name'] or trait in card['traits'] for card in cards)]
    return synergy

In [ ]:
for card in cards:
    if 'Searcher' in card['labels']:
        synergy = get_searcher_synergy(card['effect'])
        for positive_trait in synergy['positive']:
            for other_card in cards:
                if positive_trait in other_card['name'] or positive_trait in other_card['traits']:
                    G.add_edge(card['id'], other_card['id'])
        for negative_trait in synergy['negative']:
            for other_card in cards:
                if negative_trait in other_card['name'] or negative_trait in other_card['traits']:
                    if G.has_edge(card['id'], other_card['id']):
                        G.remove_edge(card['id'], other_card['id'])
    

In [41]:
def plot_card_with_neighbors(card_id):
    # Get the neighbors of the specified card
    neighbors = list(G.neighbors(card_id))
    
    # Create a subgraph with the card and its neighbors
    subgraph = G.subgraph([card_id] + neighbors)
    
    # Draw the subgraph
    pos = nx.spring_layout(subgraph)
    plt.figure(figsize=(10, 8))
    nx.draw(subgraph, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=3000, font_size=10, font_weight='bold')
    
    # Draw node labels
    labels = {node: G.nodes[node]['name'] for node in subgraph.nodes()}
    nx.draw_networkx_labels(subgraph, pos, labels, font_size=12)
    
    plt.title(f"Graph for card {G.nodes[card_id]['name']} and its neighbors")
    plt.show()

# Example usage
# plot_card_with_neighbors('OP01-016')
print(G.nodes['ST02-007'])
# print the neighbors of the card
print('has neighbors:' , len(list(G.neighbors('ST02-007'))))
print(list(G.neighbors('ST02-007')))

{'name': 'Jewelry Bonney', 'labels': ['Searcher', 'Low Cost', 'Support'], 'traits': ['Supernovas', 'Bonney Pirates'], 'effect': '[Activate:Main] (1) (Rest the designated number of DON!! in your Cost Area) Rest this Character: Look at the top 5 cards of your deck, reveal 1 {Supernovas} type card and add it to your hand. Place the remaining cards at the bottom of the deck in any order.', 'cost': 1}
has neighbors: 76
['OP01-074', 'OP01-040', 'ST02-017', 'OP01-047', 'OP01-003', 'OP01-008', 'OP01-039', 'OP01-108', 'OP01-058', 'OP01-019', 'OP01-026', 'OP01-024', 'OP01-001', 'OP01-027', 'OP01-002', 'OP01-025', 'OP01-007', 'OP01-051', 'OP01-054', 'OP01-056', 'ST01-001', 'ST01-012', 'ST01-013', 'ST01-015', 'ST02-005', 'ST02-009', 'ST02-013', 'ST02-015', 'ST02-001', 'ST02-008', 'ST02-010', 'ST02-014', 'ST02-004', 'ST02-003', 'ST02-016', 'OP02-035', 'OP02-041', 'OP02-045', 'OP02-043', 'P-003', 'P-001', 'P-007', 'P-009', 'P-017', 'P-018', 'P-022', 'P-024', 'P-029', 'OP06-076', 'EB01-012', 'EB01-01